In [ ]:
## download https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-04.parquet
## TLC Trip Record 2023.04 dataset : High Volume For-Hire Vehicle Trip Records (PARQUET)
## data decription : https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_hvfhs.pdf

In [1]:
import pandas as pd

In [3]:
df = pd.read_parquet('../../data/tlc_trip_record/fhvhv_tripdata_2023-04.parquet')
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-04-01 00:01:28,2023-04-01 00:02:41,2023-04-01 00:03:03,2023-04-01 00:15:42,174,126,5.10,...,1.82,0.00,0.0,0.00,13.83,N,N,,N,N
1,HV0003,B03404,B03404,2023-04-01 00:39:40,2023-04-01 00:41:20,2023-04-01 00:43:42,2023-04-01 01:01:08,211,163,3.11,...,3.56,2.75,0.0,9.52,24.84,N,N,,N,N
2,HV0003,B03404,B03404,2023-03-31 23:56:31,2023-04-01 00:00:20,2023-04-01 00:01:01,2023-04-01 00:09:48,222,76,2.40,...,1.08,0.00,0.0,0.00,8.10,Y,N,,N,N
3,HV0003,B03404,B03404,2023-04-01 00:12:35,2023-04-01 00:16:04,2023-04-01 00:17:17,2023-04-01 00:19:12,76,124,0.43,...,0.70,0.00,0.0,0.00,5.40,N,N,,N,N
4,HV0003,B03404,B03404,2023-04-01 00:46:23,2023-04-01 00:47:30,2023-04-01 00:47:49,2023-04-01 01:05:23,263,247,4.18,...,1.48,2.75,0.0,0.00,15.65,N,N,,N,N


In [4]:
from pyspark import SparkConf, SparkContext
import pandas as pd

In [5]:
conf = SparkConf().setMaster("local").setAppName("uber-date-trips")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/18 16:54:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

/Users/woojin/Documents/data_engineer/data-engineering-gcp/venv/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
directory = "/Users/woojin/Documents/data_engineer/data-engineering-gcp/data/tlc_trip_record"
filename = "fhvhv_tripdata_2023-04.parquet"

In [29]:
# lines = sc.textFile(f"file:///{directory}/{filename}")

df = sqlContext.read.parquet(f"{directory}/{filename}")
df.show(10)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [16]:
df.first()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2023, 4, 1, 0, 1, 28), on_scene_datetime=datetime.datetime(2023, 4, 1, 0, 2, 41), pickup_datetime=datetime.datetime(2023, 4, 1, 0, 3, 3), dropoff_datetime=datetime.datetime(2023, 4, 1, 0, 15, 42), PULocationID=174, DOLocationID=126, trip_miles=5.1, trip_time=759, base_passenger_fare=20.52, tolls=0.0, bcf=0.62, sales_tax=1.82, congestion_surcharge=0.0, airport_fee=0.0, tips=0.0, driver_pay=13.83, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N')

In [27]:
# header = df.first()
# filtered_lines = df.filter(lambda row:row != header)
# dates = filtered_lines.map(lambda x: x.split(",")[2].split(" ")[0])
# dates.countByValue()
type(df)

pyspark.sql.dataframe.DataFrame

In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local') \
    .appName('uber-date-trips') \
    .getOrCreate()

In [25]:
parquetFile = spark.read.parquet(f"{directory}/{filename}")

# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("TLC_Trip")
teenagers = spark.sql("SELECT * FROM TLC_Trip")
teenagers.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--